# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:

# read the Cities_weather_data.csv to create dataframe, previous cells are not needed to be executed as long as Cities_weather_data.csv was created
cities = pd.read_csv("Cities_weather_data.csv", encoding="utf-8")
cities.head()

,City,Latitude,Longitudes,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
0,Libenge,3.65,18.63,70.54,92,100,5.91,CD,1600651060
1,Nemuro,43.32,145.57,64.40,88,75,7.52,JP,1600651060
2,Genhe,50.78,121.52,46.96,76,100,6.96,CN,1600651060
3,Lipin Bor,60.26,37.98,48.22,89,100,47.18,RU,1600651061
4,Deputatsky,69.30,139.90,45.77,62,0,2.46,RU,1600651022


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
humidity = cities["Humidity (%)"].astype(float)
maxhumidity = humidity.max()
locations = cities[["Latitude", "Longitudes"]]

In [5]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
new_city_df = cities.loc[(cities["Max Temp (F)"] > 70) & (cities["Max Temp (F)"] < 80) & (cities["Cloudiness (%)"] == 0), :]
new_city_df = new_city_df.dropna(how='any')
new_city_df.reset_index(inplace=True)
del new_city_df['index']
new_city_df.head()

,City,Latitude,Longitudes,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
0,Wanlaweyn,2.62,44.89,72.84,84,0,20.51,SO,1600651008
1,Barbosa Ferraz,-24.03,-52.01,75.20,57,0,15.50,BR,1600651091
2,Kalemie,-5.95,29.19,72.59,67,0,12.41,CD,1600651116
3,Kiama,-34.68,150.87,79.00,67,0,13.40,AU,1600651117
4,Amarante do Maranhão,-5.57,-46.74,78.28,69,0,7.45,BR,1600651123


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotellist = []

for i in range(len(new_city_df)):
    lat = new_city_df.loc[i]['Latitude']
    lng = new_city_df.loc[i]['Longitudes']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params)
    json_data = response.json()
    try:
        hotellist.append(json_data['results'][0]['name'])
    except:
        hotellist.append("")
new_city_df["Hotel Name"] = hotellist
new_city_df = new_city_df.dropna(how='any')
new_city_df.head()

,City,Latitude,Longitudes,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date,Hotel Name
0,Wanlaweyn,2.62,44.89,72.84,84,0,20.51,SO,1600651008,Wanlaweyn
1,Barbosa Ferraz,-24.03,-52.01,75.20,57,0,15.50,BR,1600651091,Barbosa Ferraz
2,Kalemie,-5.95,29.19,72.59,67,0,12.41,CD,1600651116,Kalemie
3,Kiama,-34.68,150.87,79.00,67,0,13.40,AU,1600651117,Kiama
4,Amarante do Maranhão,-5.57,-46.74,78.28,69,0,7.45,BR,1600651123,Amarante do Maranhão


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in new_city_df.iterrows()]
locations = new_city_df[["Latitude", "Longitudes"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))